In [1]:
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", device)
def swish(x):
    return x * torch.sigmoid(x)
class FontClassifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes,dropout_prob=0.5):
        super(FontClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, num_classes)
        self.dropout = nn.Dropout(p=dropout_prob)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

device: cpu


In [19]:
import pickle
# loaded_features1 = pickle.load(open('hog_features_resize500_LoG_pca99.pkl', 'rb'))
# loaded_features1 = pickle.load(open('HoG_features_resize500_LoG_16x8_pca99.pkl', 'rb'))
loaded_features1 = pickle.load(open('..\hog_features_resize500_pca99_8x16cell_2blocks.pkl', 'rb'))
#normalize
loaded_features1 = (loaded_features1 - np.mean(loaded_features1, axis=0)) / np.std(loaded_features1, axis=0)
loaded_features = pickle.load(open('..\glc_features_resize500_LoG.pkl', 'rb'))
loaded_features=np.concatenate((loaded_features1,loaded_features),axis=1)
# loaded_features=pickle.load(open('LBP_features_resize500_LoG_16x8_pca99.pkl', 'rb'))
loaded_labels = pickle.load(open('..\labels500_pca95.pkl', 'rb'))

In [26]:
import numpy as np
# Convert the lists to NumPy arrays
y = np.array(loaded_labels)
y.shape
labels = np.array(loaded_labels)
print(labels.shape)
# labels = labels[1::2]
print(labels.shape)

(4000,)
(4000,)


In [27]:
# X_train, X_test, y_train, y_test = train_test_split(loaded_features, loaded_labels, test_size=0.2, random_state=42)
X_train=loaded_features
y_train=labels
X_train = np.array(X_train)
y_train = np.array(y_train)
# X_test = np.array(X_test)
# y_test = np.array(y_test)

In [29]:
from sklearn.preprocessing import LabelEncoder

# Convert string labels to numeric labels using LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
# y_test = label_encoder.transform(y_test)

In [30]:
X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.from_numpy(y_train).long().to(device)
# X_test = torch.from_numpy(X_test).float().to(device)
# y_test = torch.from_numpy(y_test).long().to(device)

In [31]:
X_train.shape, y_train.shape

(torch.Size([4000, 2979]), torch.Size([4000]))

In [32]:


# Define hyperparameters
input_size = X_train.shape[1]  # Size of input (number of HOG features)
hidden_size1 = 512  # Number of neurons in the first hidden layer
hidden_size2 = 256   # Number of neurons in the second hidden layer
num_classes = len(np.unique(y_train))  # Number of classes

# Create the neural network
model = FontClassifier(input_size, hidden_size1, hidden_size2, num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)

# Define data loaders
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [33]:
num_epochs = 100
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/100], Loss: 1.3933
Epoch [2/100], Loss: 1.2870
Epoch [3/100], Loss: 1.0239
Epoch [4/100], Loss: 1.0058
Epoch [5/100], Loss: 0.9357
Epoch [6/100], Loss: 0.9660
Epoch [7/100], Loss: 0.7580
Epoch [8/100], Loss: 0.7431
Epoch [9/100], Loss: 0.7710
Epoch [10/100], Loss: 0.7120
Epoch [11/100], Loss: 0.7062
Epoch [12/100], Loss: 0.5602
Epoch [13/100], Loss: 0.7329
Epoch [14/100], Loss: 0.6532
Epoch [15/100], Loss: 0.8144
Epoch [16/100], Loss: 0.6686
Epoch [17/100], Loss: 0.5220
Epoch [18/100], Loss: 0.4813
Epoch [19/100], Loss: 0.5291
Epoch [20/100], Loss: 0.4018
Epoch [21/100], Loss: 0.5020
Epoch [22/100], Loss: 0.6695
Epoch [23/100], Loss: 0.5388
Epoch [24/100], Loss: 0.4852
Epoch [25/100], Loss: 0.5706
Epoch [26/100], Loss: 0.6745
Epoch [27/100], Loss: 0.4862
Epoch [28/100], Loss: 0.6683
Epoch [29/100], Loss: 0.4186
Epoch [30/100], Loss: 0.4038
Epoch [31/100], Loss: 0.3147
Epoch [32/100], Loss: 0.3763
Epoch [33/100], Loss: 0.4924
Epoch [34/100], Loss: 0.4230
Epoch [35/100], Loss: 0

In [30]:
test_dataset = TensorDataset(X_test, y_test)

test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy: {100 * correct / total}%')

In [34]:
torch.save(model.state_dict(), 'font_classifier_model_pytorch_HoGLCM88_Deploy.pth')